### Problem Statement

I am going to build a music recommender system using the Spotify and Genius APIs. The app will sort an ablum entered in the order of songs the user will mostly likely like by analyzing the audio features and lyrics of the users recently saved tracks, recently played tracks, and top tracks.

### Library/Packages Import

In [1]:
import pandas as pd
import numpy as np
from time import sleep
from re import sub

import spotipy
from spotipy.oauth2 import SpotifyOAuth

import lyricsgenius

pd.set_option('display.max_rows', None)

### API User Authorization

In [2]:
cid = '01e5c3c39a334ae78bf5becf053ad2d5'
secret = '3c481468946e43dc9da43ed6b5c16bc8'
uri = 'http://localhost:9999/lab/workspaces/auto-c'
cache = '.spotipyoauthcache'
scope = 'user-read-recently-played user-library-read user-top-read'

# 'user-read-recently-played' --> Get Current User's Recently Played Tracks
# 'user-top-read' --> Get a User's Top Artists and Tracks
# 'user-library-read' --> Get a User's Saved Tracks

In [3]:
auth = SpotifyOAuth(client_id = cid,
                    client_secret = secret,
                    redirect_uri = uri,
                    cache_path = cache,
                    scope = scope)

sp = spotipy.Spotify(oauth_manager = auth)
sp.current_user()

{'display_name': 'Aziz Maredia',
 'external_urls': {'spotify': 'https://open.spotify.com/user/1282060363'},
 'followers': {'href': None, 'total': 36},
 'href': 'https://api.spotify.com/v1/users/1282060363',
 'id': '1282060363',
 'images': [{'height': None,
   'url': 'https://scontent-ort2-1.xx.fbcdn.net/v/t1.0-1/p320x320/79640598_10158654118142079_1999166169432457216_n.jpg?_nc_cat=104&ccb=2&_nc_sid=0c64ff&_nc_ohc=30hlePYNqz8AX_eb0dh&_nc_ht=scontent-ort2-1.xx&tp=6&oh=548d9a1b6b45469ec9cbd24f5b8bee09&oe=601C18A9',
   'width': None}],
 'type': 'user',
 'uri': 'spotify:user:1282060363'}

### User Top Tracks

In [4]:
def get_top_tracks(var = None):
 
    df = None
    
    for i in range(0, 60, 20):
        
        results = sp.current_user_top_tracks(offset = i)

        top_tracks_dict = {}

        top_tracks_dict['track'] = [i['name'] for i in results['items']]
        top_tracks_dict['artist'] = [i['artists'][0]['name'] for i in results['items']]
        top_tracks_dict['album'] = [i['album']['name'] for i in results['items']]
        top_tracks_dict['popularity_score'] = [i['popularity'] for i in results['items']]
        top_tracks_dict['track_id'] = [i['id'] for i in results['items']]
        top_tracks_dict['track_uri'] = [i['uri'] for i in results['items']]
        top_tracks_dict['spotify_link'] = [i['external_urls']['spotify'] for i in results['items']]
        top_tracks_dict['preview'] = [i['preview_url'] for i in results['items']]
        top_tracks_dict['album_image_large'] = [i['album']['images'][1]['url'] for i in results['items']]
        top_tracks_dict['album_image_small'] = [i['album']['images'][2]['url'] for i in results['items']]
    
        if i == 0:
            df = pd.DataFrame(top_tracks_dict)
        
        else:
            df = pd.concat([df, pd.DataFrame(top_tracks_dict)])
            df.reset_index(drop = True, inplace = True)
    
    return df

### User Current Saved

In [5]:
def get_current_saved(var = None):
    
    df = None
    
    for i in range(0, 200, 20):
    
        results = sp.current_user_saved_tracks(offset = i)

        current_saved_dict = {}

        current_saved_dict['track'] = [i['track']['name'] for i in results['items']]
        current_saved_dict['artist'] = [i['track']['artists'][0]['name'] for i in results['items']]
        current_saved_dict['album'] = [i['track']['album']['name'] for i in results['items']]
        current_saved_dict['popularity_score'] = [i['track']['popularity'] for i in results['items']]
        current_saved_dict['track_id'] = [i['track']['id'] for i in results['items']]
        current_saved_dict['track_uri'] = [i['track']['uri'] for i in results['items']]
        current_saved_dict['spotify_link'] = [i['track']['external_urls']['spotify'] for i in results['items']]
        current_saved_dict['preview'] = [i['track']['preview_url'] for i in results['items']]
        current_saved_dict['album_image_large'] = [i['track']['album']['images'][1]['url'] for i in results['items']]
        current_saved_dict['album_image_small'] = [i['track']['album']['images'][2]['url'] for i in results['items']]
    
        if i == 0:
            df = pd.DataFrame(current_saved_dict)
        
        else:
            df = pd.concat([df, pd.DataFrame(current_saved_dict)])
            df.reset_index(drop = True, inplace = True)
    
    return df

### User Recently Played

In [6]:
def get_recent_played(var = None):
    
    results = sp.current_user_recently_played()

    recent_played_dict = {}

    recent_played_dict['track'] = [i['track']['name'] for i in results['items']]
    recent_played_dict['artist'] = [i['track']['artists'][0]['name'] for i in results['items']]
    recent_played_dict['album'] = [i['track']['album']['name'] for i in results['items']]
    recent_played_dict['popularity_score'] = [i['track']['popularity'] for i in results['items']]
    recent_played_dict['track_id'] = [i['track']['id'] for i in results['items']]
    recent_played_dict['track_uri'] = [i['track']['uri'] for i in results['items']]
    recent_played_dict['spotify_link'] = [i['track']['external_urls']['spotify'] for i in results['items']]
    recent_played_dict['preview'] = [i['track']['preview_url'] for i in results['items']]
    recent_played_dict['album_image_large'] = [i['track']['album']['images'][1]['url'] for i in results['items']]
    recent_played_dict['album_image_small'] = [i['track']['album']['images'][2]['url'] for i in results['items']]
    
    return pd.DataFrame(recent_played_dict).drop_duplicates().reset_index(drop = True)

### Get Tracks w/ Audio Features

In [7]:
def get_tracks_w_audio_features(var = None):
    
    all_tracks_df = pd.concat([get_top_tracks(), get_current_saved(), get_recent_played()], axis = 0)
    all_tracks_df.drop_duplicates(inplace = True)
    all_tracks_df.reset_index(drop = True, inplace = True)
    
    df = None
    start = 0
    end = 100
    
    for i in range((len(all_tracks_df['track_id']) // 100) + 1):
    
        if i == (len(all_tracks_df['track_id']) // 100):
            results = sp.audio_features(tracks = list(all_tracks_df['track_id'])[start:len(all_tracks_df['track_id'])])
        else: 
            results = sp.audio_features(tracks = list(all_tracks_df['track_id'])[start:end])

        audio_dict = {}
        keep_features = ['danceability', 'energy', 'key', 'loudness', 'mode',
                         'speechiness', 'acousticness', 'instrumentalness', 'liveness',
                         'valence', 'tempo', 'id', 'duration_ms', 'time_signature']

        for i in keep_features:
            audio_dict[i] = [x[i] for x in results]
        
        if i == 0:
            df = pd.DataFrame(audio_dict)
        else:
            df = pd.concat([df, pd.DataFrame(audio_dict)])
            df.reset_index(drop = True, inplace = True)
        
        start += 100
        end += 100
    
    return pd.merge(all_tracks_df, df.rename(columns = {'id':'track_id'}), on = 'track_id', how = 'inner')

In [8]:
user_tracks_df = get_tracks_w_audio_features()
user_tracks_df.head()

,track,artist,album,popularity_score,track_id,track_uri,spotify_link,preview,album_image_large,album_image_small,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,ROXANNE,Arizona Zervas,ROXANNE,89,696DnlkuDOXcMAnKlTgXXK,spotify:track:696DnlkuDOXcMAnKlTgXXK,https://open.spotify.com/track/696DnlkuDOXcMAn...,https://p.scdn.co/mp3-preview/17c8499c72603f1a...,https://i.scdn.co/image/ab67616d00001e02069a93...,https://i.scdn.co/image/ab67616d00004851069a93...,...,-5.616,0,0.1480,0.0522,0.000000,0.4600,0.457,116.735,163636,5
1,Thank You,Dido,No Angel,72,3yUcJwYu7fXAfqMj9krY6l,spotify:track:3yUcJwYu7fXAfqMj9krY6l,https://open.spotify.com/track/3yUcJwYu7fXAfqM...,https://p.scdn.co/mp3-preview/61c1b2d6f13f53af...,https://i.scdn.co/image/ab67616d00001e023e5cbf...,https://i.scdn.co/image/ab67616d000048513e5cbf...,...,-9.942,0,0.0424,0.3000,0.000215,0.0665,0.765,79.984,218360,4
2,Climbing a Wall,Floral,Floral EP,40,2GZLWeDxVyCxDBdVzVjbMi,spotify:track:2GZLWeDxVyCxDBdVzVjbMi,https://open.spotify.com/track/2GZLWeDxVyCxDBd...,https://p.scdn.co/mp3-preview/e9074cec5dbf81b0...,https://i.scdn.co/image/ab67616d00001e024ffed7...,https://i.scdn.co/image/ab67616d000048514ffed7...,...,-5.853,1,0.0436,0.0177,0.286000,0.0890,0.678,151.097,219266,1
3,BLIND (feat. Young Thug),DaBaby,BLAME IT ON BABY (DELUXE),79,2T5NBwKRySiCR78vVk08vr,spotify:track:2T5NBwKRySiCR78vVk08vr,https://open.spotify.com/track/2T5NBwKRySiCR78...,https://p.scdn.co/mp3-preview/0d8eb3d8a247f65d...,https://i.scdn.co/image/ab67616d00001e02e818d0...,https://i.scdn.co/image/ab67616d00004851e818d0...,...,-6.503,0,0.3460,0.0115,0.000000,0.0971,0.754,162.973,153467,4
4,Tequila Shots,Kid Cudi,Man On The Moon III: The Chosen,86,30KctD1WsHKTIYczXjip5a,spotify:track:30KctD1WsHKTIYczXjip5a,https://open.spotify.com/track/30KctD1WsHKTIYc...,https://p.scdn.co/mp3-preview/eea2485b714dffab...,https://i.scdn.co/image/ab67616d00001e026f43a6...,https://i.scdn.co/image/ab67616d000048516f43a6...,...,-7.214,0,0.0531,0.0840,0.000050,0.5270,0.220,90.494,193293,4


### Song Lyrics - Genius API

In [9]:
genius_cid = 'CcpHS1g5r7-WS44TjJ3sj6ArKwxRIOy8uBvBDMBR0nSqNqm5KrkR83hkoOkxBZpA'
genius_secret = 'ZnKfkN0tdVp8__7WTSkY4QnLtJaPz44lmG7iLjlnIXdLuZZrh0Bqud6FdERuq3uauroUBJGsnQqQCJOgnqwq7A'
token = '4rG9_aw6EWk-qxxhSwLhgoLi7KjjNJm-kERUuakfWtJQZLYr5pbLLE0mt8_5vBV7'

genius = lyricsgenius.Genius(token)

genius.remove_section_headers = True
genius.verbose = False

In [10]:
def get_lyrics(df):

    lyrics_lst = []

    for i in range(len(df.index)):
        song = genius.search_song(list(df['track'])[i], list(df['artist'])[i])

        try:
            if list(df['track'])[i] == song.title or list(df['artist'])[i] == song.artist:
                lyrics_lst.append(sub('\n', ' ', song.lyrics).lower())
            else:
                lyrics_lst.append(np.nan)
        except:
            lyrics_lst.append(np.nan)

        sleep(0.5)

    return lyrics_lst

In [11]:
%%time
user_tracks_df['lyrics'] = get_lyrics(user_tracks_df)

CPU times: user 14.6 s, sys: 1.15 s, total: 15.8 s
Wall time: 8min 48s


In [12]:
user_tracks_df.isnull().sum()

track                 0
artist                0
album                 0
popularity_score      0
track_id              0
track_uri             0
spotify_link          0
preview              16
album_image_large     0
album_image_small     0
danceability          0
energy                0
key                   0
loudness              0
mode                  0
speechiness           0
acousticness          0
instrumentalness      0
liveness              0
valence               0
tempo                 0
duration_ms           0
time_signature        0
lyrics               80
dtype: int64

In [13]:
user_tracks_df.head(10)

,track,artist,album,popularity_score,track_id,track_uri,spotify_link,preview,album_image_large,album_image_small,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics
0,Climbing a Wall,Floral,Floral EP,39,2GZLWeDxVyCxDBdVzVjbMi,spotify:track:2GZLWeDxVyCxDBdVzVjbMi,https://open.spotify.com/track/2GZLWeDxVyCxDBd...,https://p.scdn.co/mp3-preview/e9074cec5dbf81b0...,https://i.scdn.co/image/ab67616d00001e024ffed7...,https://i.scdn.co/image/ab67616d000048514ffed7...,...,1,0.0436,0.017700,0.286000,0.0890,0.678,151.097,219266,1,NaN
1,ROXANNE,Arizona Zervas,ROXANNE,88,696DnlkuDOXcMAnKlTgXXK,spotify:track:696DnlkuDOXcMAnKlTgXXK,https://open.spotify.com/track/696DnlkuDOXcMAn...,https://p.scdn.co/mp3-preview/17c8499c72603f1a...,https://i.scdn.co/image/ab67616d00001e02069a93...,https://i.scdn.co/image/ab67616d00004851069a93...,...,0,0.1480,0.052200,0.000000,0.4600,0.457,116.735,163636,5,"all for the 'gram bitches love the 'gram oh, w..."
2,I'm God,Clams Casino,Instrumental Relics,63,0FVuyC9RP5MACjp4lgU3qZ,spotify:track:0FVuyC9RP5MACjp4lgU3qZ,https://open.spotify.com/track/0FVuyC9RP5MACjp...,https://p.scdn.co/mp3-preview/b3b4e58ac20fe9b9...,https://i.scdn.co/image/ab67616d00001e029dd5e3...,https://i.scdn.co/image/ab67616d000048519dd5e3...,...,1,0.0336,0.071100,0.760000,0.1020,0.089,141.927,277711,4,NaN
3,Thank You,Dido,No Angel,71,3yUcJwYu7fXAfqMj9krY6l,spotify:track:3yUcJwYu7fXAfqMj9krY6l,https://open.spotify.com/track/3yUcJwYu7fXAfqM...,https://p.scdn.co/mp3-preview/61c1b2d6f13f53af...,https://i.scdn.co/image/ab67616d00001e023e5cbf...,https://i.scdn.co/image/ab67616d000048513e5cbf...,...,0,0.0424,0.300000,0.000215,0.0665,0.765,79.984,218360,4,"my tea's gone cold, i'm wondering why i got ou..."
4,BLIND (feat. Young Thug),DaBaby,BLAME IT ON BABY (DELUXE),79,2T5NBwKRySiCR78vVk08vr,spotify:track:2T5NBwKRySiCR78vVk08vr,https://open.spotify.com/track/2T5NBwKRySiCR78...,https://p.scdn.co/mp3-preview/0d8eb3d8a247f65d...,https://i.scdn.co/image/ab67616d00001e02e818d0...,https://i.scdn.co/image/ab67616d00004851e818d0...,...,0,0.3460,0.011500,0.000000,0.0971,0.754,162.973,153467,4,"yeah, yeah, yeah i've been blind for a while n..."
5,Opaul,Freddie Dredd,Pink Lotus,66,4areFtLMjGt4GmO4IAbGo0,spotify:track:4areFtLMjGt4GmO4IAbGo0,https://open.spotify.com/track/4areFtLMjGt4GmO...,https://p.scdn.co/mp3-preview/3ef6e974469be45d...,https://i.scdn.co/image/ab67616d00001e02d03785...,https://i.scdn.co/image/ab67616d00004851d03785...,...,1,0.2130,0.005550,0.498000,0.0983,0.638,145.031,140488,4,"oh david, não vai não agora que esse som tá fi..."
6,Roses Remix (feat. Future),SAINt JHN,Roses Remix (feat. Future),71,1KyZ4rUiW2lkqGNLQYKpcC,spotify:track:1KyZ4rUiW2lkqGNLQYKpcC,https://open.spotify.com/track/1KyZ4rUiW2lkqGN...,https://p.scdn.co/mp3-preview/af9080fbaac455c0...,https://i.scdn.co/image/ab67616d00001e021dec94...,https://i.scdn.co/image/ab67616d000048511dec94...,...,1,0.1130,0.024700,0.000055,0.1920,0.312,127.003,187898,4,"remix off the corner with the money, i got a b..."
7,Many Men,21 Savage,SAVAGE MODE II,75,3CDVMejYHnB1SkEEx0T1N4,spotify:track:3CDVMejYHnB1SkEEx0T1N4,https://open.spotify.com/track/3CDVMejYHnB1SkE...,https://p.scdn.co/mp3-preview/850704c41a453d9a...,https://i.scdn.co/image/ab67616d00001e02aa5790...,https://i.scdn.co/image/ab67616d00004851aa5790...,...,0,0.0322,0.000647,0.747000,0.1130,0.281,77.594,201856,4,"metro ayy, ayy, ayy, ayy, ayy, ayy i kept it ..."
8,"WHATS POPPIN (feat. DaBaby, Tory Lanez & Lil W...",Jack Harlow,"WHATS POPPIN (feat. DaBaby, Tory Lanez & Lil W...",88,2MbdDtCv5LUVjYy9RuGTgC,spotify:track:2MbdDtCv5LUVjYy9RuGTgC,https://open.spotify.com/track/2MbdDtCv5LUVjYy...,https://p.scdn.co/mp3-preview/3748f897bbea6068...,https://i.scdn.co/image/ab67616d00001e02b5f0a8...,https://i.scdn.co/image/ab67616d00004851b5f0a8...,...,0,0.2600,0.063100,0.000000,0.1850,0.835,145.013,227478,4,back with the remix these boys all my sons lik...
9,Do My Dance,Tyga,Do My Dance,56,7Hk12JkCFwLO

### Export Tracks Dataframe

In [14]:
user_tracks_df.to_csv('./datasets/aziz_tracks.csv', index = False)

In [14]:
results = sp.search('Issa Album', 1, 0, 'album', None)
album_id = results['albums']['items'][0]['id']
album_name = results['albums']['items'][0]['name']
album_image_large = results['albums']['items'][0]['images'][1]['url']
album_image_small = results['albums']['items'][0]['images'][2]['url']

In [19]:
def get_album_tracks(album_id):
 
    df = None
    
    results = sp.album_tracks(album_id)

    top_tracks_dict = {}

    top_tracks_dict['track'] = [i['name'] for i in results['items']]
    top_tracks_dict['artist'] = [i['artists'][0]['name'] for i in results['items']]
#     top_tracks_dict['album'] = [i['album']['name'] for i in results['items']]
#     top_tracks_dict['popularity_score'] = [i['popularity'] for i in results['items']]
    top_tracks_dict['track_id'] = [i['id'] for i in results['items']]
    top_tracks_dict['track_uri'] = [i['uri'] for i in results['items']]
    top_tracks_dict['spotify_link'] = [i['external_urls']['spotify'] for i in results['items']]
    top_tracks_dict['preview'] = [i['preview_url'] for i in results['items']]
#     top_tracks_dict['album_image_large'] = [i['album']['images'][1]['url'] for i in results['items']]
#     top_tracks_dict['album_image_small'] = [i['album']['images'][2]['url'] for i in results['items']]
    
    df = pd.DataFrame(top_tracks_dict)
    
    df['album'] = album_name
    df['album_image_large'] = album_image_large
    df['album_image_small'] = album_image_small
    
    
    results2 = sp.tracks(tracks = list(df['track_id']))
    df['popularity_score'] = [[i][0]['popularity'] for i in results2['tracks']]
    
    df = df[['track', 'artist', 'album', 'popularity_score', 'track_id', 'track_uri', 'spotify_link', 'preview', 'album_image_large', 'album_image_small']]
    

    
    results3 = sp.audio_features(tracks = list(df['track_id']))
                                 
    audio_dict = {}
    keep_features = ['danceability', 'energy', 'key', 'loudness', 'mode',
                         'speechiness', 'acousticness', 'instrumentalness', 'liveness',
                         'valence', 'tempo', 'id', 'duration_ms', 'time_signature']

    for i in keep_features:
        audio_dict[i] = [x[i] for x in results3]
        
    df2 = pd.DataFrame(audio_dict)
    
                                 
    return pd.merge(df, df2.rename(columns = {'id':'track_id'}), on = 'track_id', how = 'inner')

In [20]:
album = get_album_tracks(album_id)

In [21]:
album

,track,artist,album,popularity_score,track_id,track_uri,spotify_link,preview,album_image_large,album_image_small,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Bank Account,21 Savage,Issa Album,81,2fQrGHiQOvpL9UgPvtYy6G,spotify:track:2fQrGHiQOvpL9UgPvtYy6G,https://open.spotify.com/track/2fQrGHiQOvpL9Ug...,https://p.scdn.co/mp3-preview/249295a31e99403d...,https://i.scdn.co/image/ab67616d00001e02ac3c0a...,https://i.scdn.co/image/ab67616d00004851ac3c0a...,...,-8.228,0,0.351,0.015100,0.000007,0.0871,0.3760,75.016,220307,4
1,Numb,21 Savage,Issa Album,65,1vc2YF7ZvUtwr45HNpdGch,spotify:track:1vc2YF7ZvUtwr45HNpdGch,https://open.spotify.com/track/1vc2YF7ZvUtwr45...,https://p.scdn.co/mp3-preview/5f079a4df65d4d33...,https://i.scdn.co/image/ab67616d00001e02ac3c0a...,https://i.scdn.co/image/ab67616d00004851ac3c0a...,...,-11.344,0,0.285,0.043800,0.000003,0.1310,0.4640,82.008,271307,4
2,FaceTime,21 Savage,Issa Album,58,55GLc4nywcX4aIlOcx1u06,spotify:track:55GLc4nywcX4aIlOcx1u06,https://open.spotify.com/track/55GLc4nywcX4aIl...,https://p.scdn.co/mp3-preview/732861c004a6e769...,https://i.scdn.co/image/ab67616d00001e02ac3c0a...,https://i.scdn.co/image/ab67616d00004851ac3c0a...,...,-11.115,1,0.306,0.219000,0.000000,0.1070,0.3830,93.024,239160,4
3,Bad Business,21 Savage,Issa Album,55,5ZIZgRrrDj60O0F5oA9bVz,spotify:track:5ZIZgRrrDj60O0F5oA9bVz,https://open.spotify.com/track/5ZIZgRrrDj60O0F...,https://p.scdn.co/mp3-preview/8dc19bc1b2a73392...,https://i.scdn.co/image/ab67616d00001e02ac3c0a...,https://i.scdn.co/image/ab67616d00004851ac3c0a...,...,-9.729,0,0.373,0.000985,0.000016,0.0854,0.3870,158.050,162253,4
4,Whole Lot,21 Savage,Issa Album,59,2aNdXtCtyoB2KrUh7pJYxO,spotify:track:2aNdXtCtyoB2KrUh7pJYxO,https://open.spotify.com/track/2aNdXtCtyoB2KrU...,https://p.scdn.co/mp3-preview/cf74e5712c21b4ab...,https://i.scdn.co/image/ab67616d00001e02ac3c0a...,https://i.scdn.co/image/ab67616d00004851ac3c0a...,...,-9.790,0,0.245,0.188000,0.002860,0.1150,0.2230,123.020,313987,4
5,Close My Eyes,21 Savage,Issa Album,53,1mxC48728CxBA7WdKw6nFB,spotify:track:1mxC48728CxBA7WdKw6nFB,https://open.spotify.com/track/1mxC48728CxBA7W...,https://p.scdn.co/mp3-preview/e39c7fe88341f27a...,https://i.scdn.co/image/ab67616d00001e02ac3c0a...,https://i.scdn.co/image/ab67616d00004851ac3c0a...,...,-7.880,0,0.430,0.164000,0.432000,0.1560,0.1980,139.996,292267,4
6,Special,21 Savage,Issa Album,54,6QMlELXFU07NLdqv3DGNPI,spotify:track:6QMlELXFU07NLdqv3DGNPI,https://open.spotify.com/track/6QMlELXFU07NLdq...,https://p.scdn.co/mp3-preview/ca45b679c1023adf...,https://i.scdn.co/image/ab67616d00001e02ac3c0a...,https://i.scdn.co/image/ab67616d00004851ac3c0a...,...,-12.813,1,0.426,0.147000,0.000019,0.1170,0.3310,136.005,217867,4
7,Baby Girl,21 Savage,Issa Album,53,3rBwHfVPzmfR3ZVSliB0Om,spotify:track:3rBwHfVPzmfR3ZVSliB0Om,https://open.spotify.com/track/3rBwHfVPzmfR3ZV...,https://p.scdn.co/mp3-preview/564f680a7ef73f7e...,https://i.scdn.co/image/ab67616d00001e02ac3c0a...,https://i.scdn.co/image/ab67616d00004851ac3c0a...,...,-9.658,1,0.391,0.007810,0.119000,0.1330,0.1570,163.021,169093,4
8,Thug Life,21 Savage,Issa Album,53,0mhsw7HRwRwF79ES5ucP9Y,spotify:track:0mhsw7HRwRwF79ES5ucP9Y,https://open.spotify.com/track/0mhsw7HRwRwF79E...,https://p.scdn.co/mp3-preview/a5f88280af59b1c4...,https://i.scdn.co/image/ab67616d00001e02ac3c0a...,https://i.scdn.co/image/ab67616d00004851ac3c0a...,...,-9.974,1,0.450,0.159000,0.000000,0.6530,0.3070,132.054,263427,4
9,Famous,21 Savage,Issa Album,55,6g0TAJ1grweRwBMLQyiZFb,spotify:track:6g0TAJ1grweRwBMLQyiZFb,https://open.spotify.com/track/6g0TAJ1grweRwBM...,https://p.scdn.co/mp3-preview/ca63c6dcea85602f...,https://i.scdn.co/image/ab67616d00001e02ac3c0a...,https://i.scdn.co/image/ab67616d00004851ac3c0a...,...,-9.652,0,0.434,0.723000,0.000045,0.0508,0.6460,173.986,234280,4


In [22]:
album['lyrics'] = get_lyrics(album)

In [23]:
album

,track,artist,album,popularity_score,track_id,track_uri,spotify_link,preview,album_image_large,album_image_small,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics
0,Bank Account,21 Savage,Issa Album,81,2fQrGHiQOvpL9UgPvtYy6G,spotify:track:2fQrGHiQOvpL9UgPvtYy6G,https://open.spotify.com/track/2fQrGHiQOvpL9Ug...,https://p.scdn.co/mp3-preview/249295a31e99403d...,https://i.scdn.co/image/ab67616d00001e02ac3c0a...,https://i.scdn.co/image/ab67616d00004851ac3c0a...,...,0,0.351,0.015100,0.000007,0.0871,0.3760,75.016,220307,4,"ooh, ooh, ooh, ow, ow, ow wow, wow, ow, ah, ow..."
1,Numb,21 Savage,Issa Album,65,1vc2YF7ZvUtwr45HNpdGch,spotify:track:1vc2YF7ZvUtwr45HNpdGch,https://open.spotify.com/track/1vc2YF7ZvUtwr45...,https://p.scdn.co/mp3-preview/5f079a4df65d4d33...,https://i.scdn.co/image/ab67616d00001e02ac3c0a...,https://i.scdn.co/image/ab67616d00004851ac3c0a...,...,0,0.285,0.043800,0.000003,0.1310,0.4640,82.008,271307,4,"grind hard, nigga, grind hard, i done grind ha..."
2,FaceTime,21 Savage,Issa Album,58,55GLc4nywcX4aIlOcx1u06,spotify:track:55GLc4nywcX4aIlOcx1u06,https://open.spotify.com/track/55GLc4nywcX4aIl...,https://p.scdn.co/mp3-preview/732861c004a6e769...,https://i.scdn.co/image/ab67616d00001e02ac3c0a...,https://i.scdn.co/image/ab67616d00004851ac3c0a...,...,1,0.306,0.219000,0.000000,0.1070,0.3830,93.024,239160,4,"mustard on the beat, hoe henny in my system, ..."
3,Bad Business,21 Savage,Issa Album,55,5ZIZgRrrDj60O0F5oA9bVz,spotify:track:5ZIZgRrrDj60O0F5oA9bVz,https://open.spotify.com/track/5ZIZgRrrDj60O0F...,https://p.scdn.co/mp3-preview/8dc19bc1b2a73392...,https://i.scdn.co/image/ab67616d00001e02ac3c0a...,https://i.scdn.co/image/ab67616d00004851ac3c0a...,...,0,0.373,0.000985,0.000016,0.0854,0.3870,158.050,162253,4,"yeah lil bitch, yeah know what goin' on lil bi..."
4,Whole Lot,21 Savage,Issa Album,59,2aNdXtCtyoB2KrUh7pJYxO,spotify:track:2aNdXtCtyoB2KrUh7pJYxO,https://open.spotify.com/track/2aNdXtCtyoB2KrU...,https://p.scdn.co/mp3-preview/cf74e5712c21b4ab...,https://i.scdn.co/image/ab67616d00001e02ac3c0a...,https://i.scdn.co/image/ab67616d00004851ac3c0a...,...,0,0.245,0.188000,0.002860,0.1150,0.2230,123.020,313987,4,"yeah nigga, it's a knife and all, y'know what ..."
5,Close My Eyes,21 Savage,Issa Album,53,1mxC48728CxBA7WdKw6nFB,spotify:track:1mxC48728CxBA7WdKw6nFB,https://open.spotify.com/track/1mxC48728CxBA7W...,https://p.scdn.co/mp3-preview/e39c7fe88341f27a...,https://i.scdn.co/image/ab67616d00001e02ac3c0a...,https://i.scdn.co/image/ab67616d00004851ac3c0a...,...,0,0.430,0.164000,0.432000,0.1560,0.1980,139.996,292267,4,"lot of shots (yeah, yeah) lot of shots (lot of..."
6,Special,21 Savage,Issa Album,54,6QMlELXFU07NLdqv3DGNPI,spotify:track:6QMlELXFU07NLdqv3DGNPI,https://open.spotify.com/track/6QMlELXFU07NLdq...,https://p.scdn.co/mp3-preview/ca45b679c1023adf...,https://i.scdn.co/image/ab67616d00001e02ac3c0a...,https://i.scdn.co/image/ab67616d00004851ac3c0a...,...,1,0.426,0.147000,0.000019,0.1170,0.3310,136.005,217867,4,"in my feelin's, she got me in my feelins she g..."
7,Baby Girl,21 Savage,Issa Album,53,3rBwHfVPzmfR3ZVSliB0Om,spotify:track:3rBwHfVPzmfR3ZVSliB0Om,https://open.spotify.com/track/3rBwHfVPzmfR3ZV...,https://p.scdn.co/mp3-preview/564f680a7ef73f7e...,https://i.scdn.co/image/ab67616d00001e02ac3c0a...,https://i.scdn.co/image/ab67616d00004851ac3c0a...,...,1,0.391,0.007810,0.119000,0.1330,0.1570,163.021,169093,4,"yeah, whole pointers 'round my muhfuckin' neck..."
8,Thug Life,21 Savage,Issa Album,53,0mhsw7HRwRwF79ES5ucP9Y,spotify:track:0mhsw7HRwRwF79ES5ucP9Y,https://open.spotify.com/track/0mhsw7HRwRwF79E...,https://p.scdn.co/mp3-preview/a5f88280af59b1c4...,https://i.scdn.co/image/ab67616d00001e02ac3c0a...,https://i.scdn.co/image/ab67616d00004851ac3c0a...,...,1,0.450,0.159000,0.000000,0.6530,0.3070,132.054,263427,4,"scrape the corner, buy a rover crack inside my..."
9,Famous,21 Savage,Issa Album,55,6g0TAJ1grweRwBMLQyiZFb,spotify:track:6g0TAJ1grweRwBMLQyiZFb,https:/

In [24]:
album.shape

(14, 24)

In [25]:
user_tracks_df.shape

(209, 23)

In [27]:
album.to_csv('.././datasets/album.csv', index = False)